In [26]:
import gradio as gr
import pandas as pd
import numpy as np

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS

In [2]:
encoded_stock_data_path = '../data/stock_data_encoded_no_duplicates.csv'
stock_data = pd.read_csv(encoded_stock_data_path)

In [4]:
stock_data.head(2)

,symbol,name,buisnesssummary,gics_#NAME?,gics_45103010 - Application Software,gics_Automobiles & Components,gics_Banks,gics_Banks Finance & Insurance,gics_Capital Goods,gics_Capital Goods.,...,gics_Power and Energy,gics_Real Estate,gics_Real Estate (6010),gics_Real Estate Development,gics_Retailing,gics_Services,gics_Telecommunication Services,gics_Trading,gics_Transpotation,gics_Utilities
0,ABAN,ABANS ELECTRICALS PLC,Manufacturing & Assembling Household Electric ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AFSL,ABANS FINANCE PLC,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
stock_data = stock_data.fillna('unknown')

In [6]:
stock_data['symbol'] = stock_data['symbol'].str.replace(r"(","")
stock_data['symbol'] = stock_data['symbol'].str.replace(r")","")

C:\Users\bpadmin\AppData\Local\Temp\ipykernel_14984\996691076.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stock_data['symbol'] = stock_data['symbol'].str.replace(r"(","")
C:\Users\bpadmin\AppData\Local\Temp\ipykernel_14984\996691076.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stock_data['symbol'] = stock_data['symbol'].str.replace(r")","")


In [8]:
embeddings = HuggingFaceEmbeddings()

modules.json: 100%|██████████| 349/349 [00:00<00:00, 70.2kB/s]
c:\Users\bpadmin\anaconda3\envs\tf_tr_recommender\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bpadmin\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sen

In [9]:
stock_meta_data = [{'symbol': stock_data.iloc[idx]['symbol'], 'name' : stock_data.iloc[idx]['name']} for idx in range(stock_data.shape[0])]
stock_meta_data

[{'symbol': 'ABAN', 'name': 'ABANS ELECTRICALS PLC'},
 {'symbol': 'AFSL', 'name': 'ABANS FINANCE PLC'},
 {'symbol': 'AEL', 'name': 'ACCESS ENGINEERING PLC'},
 {'symbol': 'ACL', 'name': 'ACL CABLES PLC'},
 {'symbol': 'APLA', 'name': 'ACL PLASTICS PLC'},
 {'symbol': 'ACME', 'name': 'ACME PRINTING & PACKAGING PLC'},
 {'symbol': 'AGAL', 'name': 'AGALAWATTE  PLANTATIONS  PLC'},
 {'symbol': 'AGST', 'name': 'AGSTAR PLC'},
 {'symbol': 'AHUN', 'name': 'AITKEN SPENCE HOTEL HOLDINGS PLC'},
 {'symbol': 'ASPM', 'name': 'AITKEN SPENCE PLANTATION MANAGEMENTS PLC'},
 {'symbol': 'SPEN', 'name': 'AITKEN SPENCE PLC'},
 {'symbol': 'ALLI', 'name': 'ALLIANCE FINANCE COMPANY PLC'},
 {'symbol': 'AFS', 'name': 'ALPHA FIRE SERVICES PLC'},
 {'symbol': 'ALUM', 'name': 'ALUMEX PLC'},
 {'symbol': 'ATL', 'name': 'AMANA TAKAFUL PLC'},
 {'symbol': 'ATLL', 'name': 'AMANA TAKAFUL LIFE PLC'},
 {'symbol': 'TAP', 'name': 'AMBEON CAPITAL PLC'},
 {'symbol': 'GREG', 'name': 'AMBEON HOLDINGS PLC'},
 {'symbol': 'AMCL', 'name': 

In [10]:
originals = [ stock_data.iloc[idx]['name'] + ' ' + stock_data.iloc[idx]['buisnesssummary'] for idx in range(stock_data.shape[0])]
originals[:5]

['ABANS ELECTRICALS PLC Manufacturing & Assembling Household Electric & Electronic Appliances & Providing Repairs & Maintenance Services.',
 'ABANS FINANCE PLC unknown',
 'ACCESS ENGINEERING PLC unknown',
 'ACL CABLES PLC Manufacturing and Marketing of Cables and Conductors.',
 'ACL PLASTICS PLC Manufacturing of Cable grade PVC Compound.']

In [11]:
from langchain_community.vectorstores import FAISS

In [12]:
vectors = embeddings.embed_documents(originals)

In [13]:
'''Tuple[str, List[float]]'''

stock_data_embeddings = [(originals[idx],vectors[idx]) for idx in range(len(vectors))]

In [16]:
db = FAISS.from_embeddings(text_embeddings = stock_data_embeddings, embedding = embeddings, metadatas= stock_meta_data)

In [17]:
db.save_local("../vector_db/faiss_index")


In [18]:
new_db = FAISS.load_local("../vector_db/faiss_index", embeddings)

In [22]:
def get_recommendations(query, top_k):
  relevant_stocks = new_db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]
  for idx, data in enumerate(recommendations):
    print('{}. {} : {}'.format(idx+1, data[0], data[1]))


In [23]:
stock_pref = 'businesses in the domain of shipping and logistics'

get_recommendations(stock_pref, top_k = 10)

1. MSL : MERCANTILE SHIPPING COMPANY PLC
2. CABO : CARGO BOAT DEVELOPMENT COMPANY PLC
3. DOCK : COLOMBO DOCKYARD PLC
4. EMER : EASTERN MERCHANTS PLC
5. JKH : JOHN KEELLS HOLDINGS PLC
6. EAST : EAST WEST PROPERTIES PLC
7. HELA : HELA APPAREL HOLDINGS PLC
8. SEMB : S M B LEASING PLC
9. IDL : SERENDIB ENGINEERING GROUP PLC
10. DIMO : DIESEL & MOTOR ENGINEERING PLC


In [24]:
def quey_generator(domains):
    query = 'business that belong in the domains of '+','.join(domains)
    return query

def get_recommendations_gradio(domains, top_k = 10):
  query = quey_generator(domains)
  relevant_stocks = db.similarity_search_with_relevance_scores(query= query, k = top_k)
  recommendations = [(relevant_stocks[idx][0].metadata.get('symbol'), relevant_stocks[idx][0].metadata.get('name')) for idx in range(len(relevant_stocks))]

  reco_list = []
  for idx, data in enumerate(recommendations):
    reco_list.append('{}. {} : {}'.format(idx+1, data[0], data[1]))

  return_str = '\n'.join(reco_list)

  return return_str

In [27]:
demo = gr.Interface(
    fn = get_recommendations_gradio,
    inputs = gr.Dropdown(
            ["finance", "transport", "technology","agriculture"],
              multiselect=True,
             label="Domains",
             allow_custom_value = True,
             scale=5
        ),
    outputs = ["text"],
)
demo.launch(share = True)

# businesses in the domain of shipping and logistics

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
